# Notebook for additional measurements

This notebook is mostly used for debugging or validation.

## Imports

In [148]:
from pathlib import Path
from datetime import date
import os
import subprocess
import datetime
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment, Extern_Signal, Extern_Alphanumeric_Signal, Value

## Set Experiment Name

In [ ]:
experiment_name = "experiment_top_candidates_nn_complete_raw" # TODO: Set experiment name
external_signal = True # TODO: Set to True if signal is integer
external_alphanumeric_signal = True # TODO: Set to True if signal is alphanumeric
external_signal_name = "Speed" # Set name of the external_signal
external_alphanumeric_signal_name = "Gear" # Set name of external_alphanumeric_signal

experiment_folder = "../../data/experiments/taycan/gear_selected_taycan/2025-04-03_including_D2"
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

experiment_file = os.path.join(experiment_folder, f"{experiment_name}.json")
experiment = Experiment.load(experiment_file)

!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## 1. Single Measurement
Read data from a DID without saving it to an experiment file.

In [ ]:
serverid, did = (16403, 64785) # TODO: Set ServerId and DID

from revcan.signal_discovery.utils.doipclient import DoIPClient
from revcan.signal_discovery.utils.doipclient.connectors import DoIPClientUDSConnector
from revcan.signal_discovery.utils.udsoncan.client import Client

timeout = 10
ecu_ip_address = "192.168.178.27"
client_logical_address = experiment.car.arb_id_pairs[0].client_logical_address
doip_client = DoIPClient(ecu_ip_address=ecu_ip_address, initial_ecu_logical_address=serverid,
                                                client_logical_address=client_logical_address)
conn = DoIPClientUDSConnector(doip_client)

try:
    with Client(conn, request_timeout=timeout) as client:
        response = client.read_data_by_identifier_first(didlist=[did])
        value_response = Value(time=datetime.datetime.now(), value=response)
        print(f"ServerID: {serverid}, DID: {did}, Response: {value_response.value}")
except Exception as e:
    print(f"An issue occurred while probing DID 0x{did:04x} for server 0x{serverid:04x}: {e}")

2. Add Measurements to Experiment

This also includes the ground truth.

In [ ]:
# TODO: Set ground truth value and number of sample
ground_truth_alphanumeric = "D2"
ground_truth_int = 100
num_of_samples = 5

# Load Experiment for Sanity Checks
experiment = Experiment.load(experiment_file)

# Sanity check for external signal name
if external_alphanumeric_signal:
    if not experiment.external_alphanumeric_measurements[0].name == external_alphanumeric_signal_name:
        raise SystemExit(f"Name of external signal in experiment not equal to set external signal name, experiment.external_alphanumeric_measurements[0].name: {experiment.external_alphanumeric_measurements[0].name}, external_signal_name: {external_signal_name}")
if external_signal:
    if not experiment.external_measurements[0].name == external_signal_name:
        raise SystemExit(f"Name of external signal in experiment not equal to set external signal name, experiment.external_measurements[0].name: {experiment.external_measurements[0].name}, external_signal_name: {external_signal_name}")

# Sanity check for external signal data before read data
if external_alphanumeric_signal:
    if len(experiment.external_alphanumeric_measurements[0].values) != len(experiment.measurements[-1].values):
        raise SystemExit(f"Number of external signal values does not equal number of values before read, External alphanumeric measurements: {len(experiment.external_alphanumeric_measurements[0].values)}, Measurements: {len(experiment.measurements[-1].values)}")
if external_signal:
    if len(experiment.external_measurements[0].values) != len(experiment.measurements[-1].values):
        raise SystemExit(f"Number of external signal values does not equal number of values before read, External measurements: {len(experiment.external_measurements[0].values)}, Measurements: {len(experiment.measurements[-1].values)}")

# Read data
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{experiment_file}' --activate_logging true --num_samples {num_of_samples}

# Load Experiment again
experiment = Experiment.load(experiment_file)

# Add ground truth to experiment
if external_alphanumeric_signal:
    values_alphanumeric:List[str] = []
    for i in range(num_of_samples):
        values_alphanumeric.append(ground_truth_alphanumeric)
    experiment.external_alphanumeric_measurements[0].values += deepcopy(values_alphanumeric)
    print(f"Experiment: {len(experiment.external_alphanumeric_measurements[0].values)} ground truth values")
if external_signal:
    values:List[Value] = []
    for i in range(num_of_samples):
        values.append(Value(value=[ground_truth_int], time=datetime.datetime.now()))
    experiment.external_measurements[0].values += deepcopy(values)
    print(f"Experiment: {len(experiment.external_measurements[0].values)} ground truth values")


# Save measurements
experiment.save(f"{experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## Additional Scripts

### Show Ground Truth

In [ ]:
gears=["P","R","N","D1", "D2"]
values = {}
for gear in gears:
    values[gear] = []

for i in range(len(experiment.external_measurements[0].values)):
    gear = experiment.external_alphanumeric_measurements[0].values[i]
    values[gear].append(experiment.external_measurements[0].values[i].value[0])

for gear in gears:
    unique_values = sorted(set(values[gear]))
    print(f"Gear {gear}: {len(values[gear])} measurements: {unique_values}")

### Set External Measurements Name

In [ ]:
experiment.external_measurements = []
experiment.external_measurements.append(Extern_Signal( name = "Speed", id = 1, values = []))

experiment.external_alphanumeric_measurements = []
experiment.external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Gear", id = 1, values = []))

experiment.save(f"{experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## Remove previous measurements

### Remove last n measurements

In [ ]:
n = 5

!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

for signal in experiment.measurements:
    signal.values = signal.values[:-n] 
    print(f"ServerID {signal.serverid}, DID: {signal.did.did}, Values: {len(signal.values)}")

experiment.external_alphanumeric_measurements[0].values = experiment.external_alphanumeric_measurements[0].values[:-n]
experiment.external_measurements[0].values = experiment.external_measurements[0].values[:-n]

print(f"Experiment: {len(experiment.external_alphanumeric_measurements[0].values)} ground truth values")
print(f"Experiment: {len(experiment.external_measurements[0].values)} ground truth values")

experiment.save(f"{experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

### Remove all measurements

In [ ]:
for signal in experiment.measurements:
    signal.values = []

experiment.experiment_runtime_seconds = 0

experiment.save(f"{experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"